In [1]:
from kiara.api import KiaraAPI
kiara = KiaraAPI.instance()

In [2]:
aws_access_key_id = 'myawsaccesskey'
aws_secret_access_key = 'myawssecretaccesskey'
aws_s3_bucket = 'mys3bucket'
db_name = 'cc_test'
table_name = 'mossfon'

In [4]:
query= f"""
SELECT *
FROM {db_name}.{table_name}
WHERE url_host_registered_domain = 'mossfon.com'
LIMIT 10
"""

In [5]:
inputs = {
    "aws_access_key_id": aws_access_key_id,
    "aws_secret_access_key": aws_secret_access_key,
    "aws_s3_bucket": aws_s3_bucket,
    "query": query,
    "db_name": db_name,
    "table_name": table_name,
 }

In [6]:
query_id = kiara.run_job('onboard.run_cc_query', inputs=inputs)

In [7]:
query_id

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field         value                                                                                                                    │
│  ────────────────────────────────────────────────────                                                                                    │
│   cc_query_id   2c4f7358-b8fb-4f16-9606-76c87af347fa                                                                                     │
│                                                                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [8]:
query_id['cc_query_id'].data

'2c4f7358-b8fb-4f16-9606-76c87af347fa'

In [9]:
# TODO: check why error if the value is entered directly as a string
# "cc_query_id": '9b18ffd9-98b8-482a-9fdd-803fd7636b91'

inputs = {
    "aws_access_key_id": aws_access_key_id,
    "aws_secret_access_key": aws_secret_access_key,
    # this doesn't work in kiara at the moment "cc_query_id": '9b18ffd9-98b8-482a-9fdd-803fd7636b91',
    "cc_query_id": query_id['cc_query_id'], 
 }

In [10]:
# wait until the output of this cell displays "state: succeeded" before processing the notebook further
query_status = kiara.run_job('onboard.get_cc_query_status', inputs=inputs)
query_status.get_value_data('cc_query_status').dict_data['QueryExecution']['Status']

{'State': 'SUCCEEDED',
 'SubmissionDateTime': datetime.datetime(2023, 10, 9, 16, 50, 53, 844000, tzinfo=tzlocal()),
 'CompletionDateTime': datetime.datetime(2023, 10, 9, 16, 50, 57, 349000, tzinfo=tzlocal())}

In [11]:
query_result = kiara.run_job('onboard.get_cc_query_result', inputs=inputs)

In [12]:
inputs = {
    "cc_query_result": query_result['cc_query_result'],
 }

In [13]:
res = query_result.get_value_data('cc_query_result')

In [14]:
query_pages = kiara.run_job('onboard.get_cc_pages', inputs=inputs)

In [15]:
query_pages

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field            value                                                                                                                 │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   cc_query_pages                                                                                                                         │
│                      dict data     {                                                                                                     │
│                                      "cc_filename": [                                                                                    │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982292181.27/robotstxt/CC-MAIN-20160823195812-00184…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982292493.37/robotstxt/CC-MAIN-20160823195812-00184…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982292607.17/robotstxt/CC-MAIN-20160823195812-00184…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982295358.50/robotstxt/CC-MAIN-20160823195815-00184…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982295424.4/robotstxt/CC-MAIN-20160823195815-00184-…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982295494.5/robotstxt/CC-MAIN-20160823195815-00184-…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982297699.43/robotstxt/CC-MAIN-20160823195817-00049…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982297699.43/robotstxt/CC-MAIN-20160823195817-00184…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982298875.42/robotstxt/CC-MAIN-20160823195818-00184…     │
│                                        "crawl-data/CC-MAIN-2016-36/segments/1471982925602.40/robotstxt/CC-MAIN-20160823200845-00184…     │
│                                      ],                                                                                                  │
│                                      "web_page": [                                                                                       │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-24T11:51:39Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-24T16:06:36Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-24T17:48:39Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-26T06:35:22Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-26T10:05:35Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-26T11:52:31Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-27T02:48:43Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-27T02:48:44Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r\nWARC-Type: response\r\nWARC-Date: 2016-08-27T08:03:20Z\r\nWARC-Record-ID: <urn:…     │
│                                        "WARC/1.0\r